After obtaining ECIs from fitting, the next step is to run Monte Carlo to probe the finite temperature properties of a system.
This notebook illustrates how to obtain a phase diagram of the binary Si-Ge system.
First we start from `system.json` -> A file containing information about the `PRIM`, composition axis, paths to cluster expansion basis sets and their corresponding coefficients.
`*_Clexulator.cc` -> A `C++` file containing the basis functions of the cluster expansion.
`eci.json` -> A file containing ECIs of the basis functions.

In [2]:
import os
import json
import numpy as np
import libcasm.clexmonte as clexmonte


# read the system information into a dictionary
with open("./input_files/system.json", "r") as f:
    system_dict = json.load(f)


# construct a clexmonte.System object
os.environ['CASM_PREFIX'] = "/home/sesha/.local/miniconda3/envs/casm2.X_vanilla/lib/python3.11/site-packages/libcasm"
os.environ['CASM_CXX'] = "clang++"

system = clexmonte.System.from_dict(data=system_dict, search_path=['input_files'])

output_dir = "summary.json"

# Now you are ready to run Monte Carlo
# let's do semi-grand canonical ensemble first
calculator = clexmonte.MonteCalculator(
       method="semigrand_canonical",
       system=system,
   )

# construct default sampling fixture parameters
thermo = calculator.make_default_sampling_fixture_params(
    label="thermo",
    output_dir=str(output_dir),
)

# set lower convergence level for potential_energy
thermo.converge(quantity="potential_energy", abs=2e-3)

# set lower convergence level for param_composition("a")
thermo.converge(quantity="param_composition", abs=2e-3, component_name=["a"])

# construct the initial state (default configuration)
state, motif, motif_id = clexmonte.make_initial_state(
    calculator=calculator,
    conditions={
        "temperature": 300.0,
        "param_chem_pot": [-1.0],
    },
    min_volume=1000,
)

# Run several, w/ dependent runs
x_list = np.arange(-4.0, 0.01, step=0.5)
for x in x_list:
    state.conditions.vector_values["param_chem_pot"] = [x]
    sampling_fixture = calculator.run_fixture(
        state=state,
        sampling_fixture_params=thermo,
    )








